In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [ ]:
from sentence_transformers import SentenceTransformer
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
dataset = load_dataset("wikitext", "wikitext-103-v1",split="train")
raw_texts=dataset["text"]
sentences=[]



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
for paragraph in raw_texts:
  if paragraph.strip()!="":
    sents=sent_tokenize(paragraph)
    sentences.extend(sents)

print("Sample sentences:", sentences[:5])
print("Total sentences:", len(sentences))

Sample sentences: [' = Valkyria Chronicles III =', ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit .', 'Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable .', 'Released in January 2011 in Japan , it is the third game in the Valkyria series .', 'Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .']
Total sentences: 4290307


In [ ]:
# embeddings =t2vec_model.predict(sentences,source_lang="eng_Latn")
# embeddings.shape

In [ ]:
from tqdm import tqdm
import torch
def get_embeddings_in_batches(sentences, batch_size=32):
    all_embeddings = []
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[i:i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True)
        all_embeddings.append(batch_embeddings)
    return torch.cat(all_embeddings, dim=0)

embeddings = get_embeddings_in_batches(sentences, batch_size=256)

100%|██████████| 16760/16760 [29:19<00:00,  9.52it/s]


In [ ]:
import torch.nn.functional as F
def normalize_in_batches(embeddings, batch_size=1024):
    normalized = []
    for i in tqdm(range(0, len(embeddings), batch_size)):
        batch = embeddings[i:i + batch_size]
        batch = F.normalize(batch, p=2, dim=-1)
        normalized.append(batch)
    return torch.cat(normalized, dim=0)


In [ ]:
seq_len = 10
embeddings = embeddings[:len(embeddings) // seq_len * seq_len]
embeddings = embeddings.reshape(-1, seq_len, embeddings.shape[-1])  # Shape: [batch, seq_len, 384]

# Shift for input/target: input[:-1] predicts target[+1]
inputs = embeddings[:, :-1, :]
targets = embeddings[:, 1:, :]

In [ ]:
train_size = int(0.9 * len(inputs))
X_train, X_val = inputs[:train_size], inputs[train_size:]
y_train, y_val = targets[:train_size], targets[train_size:]

In [ ]:
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

In [ ]:
class PreNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.norm = nn.LayerNorm(input_dim)
        self.linear = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.GELU()

    def forward(self, x):
        return self.activation(self.linear(self.norm(x)))

class TransformerDecoder(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_layers, ff_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.TransformerDecoderLayer(
                d_model=hidden_dim,
                nhead=num_heads,
                dim_feedforward=ff_dim,
                dropout=dropout,
                batch_first=True,
                activation='gelu'
            ) for _ in range(num_layers)
        ])
        self.pos_embed = nn.Parameter(torch.randn(1, 1024, hidden_dim))  # max seq_len = 1024

    def forward(self, x):
        seq_len = x.shape[1]
        mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool, device=x.device), diagonal=1)
        x = x + self.pos_embed[:, :seq_len]
        for layer in self.layers:
            x = layer(x, x, tgt_mask=mask)
        return x

class PostNet(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.scale = nn.Parameter(torch.ones(output_dim))
        self.shift = nn.Parameter(torch.zeros(output_dim))

    def forward(self, x):
        return self.linear(x) * self.scale + self.shift

class BaseLCM(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=512, num_heads=8,
                 num_layers=6, ff_dim=2048):
        super().__init__()
        self.prenet = PreNet(input_dim, hidden_dim)
        self.decoder = TransformerDecoder(hidden_dim, num_heads, num_layers, ff_dim)
        self.postnet = PostNet(hidden_dim, input_dim)

    def forward(self, x):
        x = self.prenet(x)
        x = self.decoder(x)
        return self.postnet(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaseLCM().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

epochs = 5
for epoch in range(epochs):
    model.train()
    train_loss = 0

    for batch_in, batch_out in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        batch_in, batch_out = batch_in.to(device), batch_out.to(device)

        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                output = model(batch_in)
                loss = loss_fn(output, batch_out)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(batch_in)
            loss = loss_fn(output, batch_out)
            loss.backward()
            optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_in, batch_out in val_loader:
            batch_in, batch_out = batch_in.to(device), batch_out.to(device)
            output = model(batch_in)
            val_loss += loss_fn(output, batch_out).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "base_lcm_model.pth")
print("Model saved to base_lcm_model.pth")